In [157]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [158]:
import sys
import os
import h5py
import scipy.io
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import Input
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json
from sklearn.preprocessing import StandardScaler

In [159]:
def Nuclei_feat(mag):
    nuc_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/nuc_Feature/'

    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_' + mag + '.mat'
    nuc_data = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data.append(row_data)
        
    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_val_' + mag + '.mat'
    nuc_data_val = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data_val.append(row_data)        

    nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_test_' + mag + '.mat'
    nuc_data_test = []
    with h5py.File(nuc_file_name) as f:
        for column in f['feature']:
            row_data = []
            for row_number in range(len(column)):            
                row_data.append(f[column[row_number]][:])   
            nuc_data_test.append(row_data)   
            
    Nuc_train_feat = np.array(nuc_data)
    Nuc_val_feat = np.array(nuc_data_val)
    Nuc_test_feat = np.array(nuc_data_test)
    del nuc_data, nuc_data_val, nuc_data_test

    Nuc_train_feat = Nuc_train_feat.reshape(len(Nuc_train_feat), 256,256,1)
    Nuc_val_feat = Nuc_val_feat.reshape(len(Nuc_val_feat), 256,256,1)
    Nuc_test_feat = Nuc_test_feat.reshape(len(Nuc_test_feat), 256,256,1)
    
    return Nuc_train_feat, Nuc_val_feat, Nuc_test_feat


In [160]:
def pca_scaler(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data_sc = scaler.transform(data)
    return data_sc

In [161]:
def PCA_feat(mag):
    pca_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/PCA_Feature/'

    pca_file_name = pca_dir+ 'breakhis_pca_feature_' + mag + '.mat'
    pca_data = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data.append(column)
        
    pca_file_name = pca_dir+ 'breakhis_pca_feature_val_' + mag + '.mat'
    pca_data_val = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data_val.append(column)
            
    pca_file_name = pca_dir+ 'breakhis_pca_feature_test_' + mag + '.mat'
    pca_data_test = []
    with h5py.File(pca_file_name) as f:
        for column in f['feature']:
            pca_data_test.append(column)
            
    PCA_train_data = np.array(pca_data)
    PCA_val_data = np.array(pca_data_val)
    PCA_test_data = np.array(pca_data_test)

    PCA_train_sc = pca_scaler(PCA_train_data)
    PCA_val_sc = scaler.transform(PCA_val_data)
    PCA_test_sc = scaler.transform(PCA_test_data)
    del PCA_train_data, PCA_val_data, PCA_test_data

    return PCA_train_sc, PCA_val_sc, PCA_test_sc

In [162]:
def mag_conn(mag):
    train_dir = 'breast_' + mag + '_split/train'
    val_dir = 'breast_' + mag + '_split/val'
    test_dir = 'breast_' + mag + '_split/test'

    if mag == '40':
        b_train, m_train, b_test, m_test, b_val, m_val = 500, 1170, 100, 137, 137, 174
    if mag == '100':
        b_train, m_train, b_test, m_test, b_val, m_val = 515, 1237, 100, 145, 135, 188
    if mag == '200':
        b_train, m_train, b_test, m_test, b_val, m_val = 498, 1190, 100, 139, 137, 178
    if mag == '400':
        b_train, m_train, b_test, m_test, b_val, m_val = 470, 1032, 100, 124, 140, 147        
    
    n_train =  b_train + m_train
    n_val =  b_val + m_val
    n_test =  b_test + m_test

    train_labels = np.array([0] * b_train + [1] * m_train)
    val_labels = np.array([0] * b_val + [1] * m_val)
    test_labels = np.array([0] * b_test + [1] * m_test)
    return train_dir, val_dir, test_dir, n_train, n_val, n_test, train_labels, val_labels, test_labels

In [163]:
def VGG_feat(mag):
    vgg_train = np.load('fine_bottleneck_feat_train_' + mag +'.npy')
    vgg_val = np.load('fine_bottleneck_feat_val_' + mag +'.npy')
    vgg_test = np.load('fine_bottleneck_feat_test_' + mag +'.npy')
    return vgg_train, vgg_val, vgg_test

In [164]:
def merge_model(mag):
    ############################################################################################
    # Input 1 (Nuclei)
    ############################################################################################
    model_1 = Sequential()
    model_1.add(Flatten(input_shape= (256, 256, 1)))
    model_1.add(Dropout(0.5))
    model_1.add(Dense(16, activation='sigmoid'))
    model_1.add(Dropout(0.5))
    model_1.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 2 (PCA)
    ############################################################################################
    PCA_input = Input(shape=(12,))
    model_2 = Sequential()
    model_2.add(Dense(512, input_dim=12))
    model_2.add(Activation('relu'))
    model_2.add(Dropout(0.5))
    model_2.add(Dense(6))
    model_2.add(Activation('sigmoid'))
    model_2.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 3 (VGG)
    ############################################################################################
    model_3 = Sequential()
    model_3.add(Flatten(input_shape= (7, 7, 512)))
    model_3.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

    merge_layer = concatenate([model_1.output, model_2.output, model_3.output])
    dnn_output = Dense(1, activation="sigmoid")(merge_layer)
    model_concat = Model(inputs=[model_1.input, model_2.input, model_3.input], outputs=dnn_output)
    return model_concat

In [165]:
def train_merge_model(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    Nuc_train, Nuc_val, Nuc_test = Nuclei_feat(mag)
    PCA_train, PCA_val, PCA_test = PCA_feat(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)

    model = merge_model(mag)
    model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
    
    model.fit(x = [Nuc_train, PCA_train, VGG_train], y = train_labels,
             validation_data = ([Nuc_val, PCA_val, VGG_val], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)
    
    test_score = model.evaluate(x = [Nuc_test, PCA_test, VGG_test], y = test_labels)
    print('\n', 'BreakHis Test accuracy:', test_score[1])

    model_new_name = 'Final_concat_Model_' + mag
    model_json = model.to_json()
    with open(model_new_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_new_name + '.h5')
    return model

In [166]:
batch_size = 5
epochs = 20
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1670 samples, validate on 311 samples
Epoch 1/20
1670/1670 [==============================] - 5s 3ms/step - loss: 0.6713 - accuracy: 0.6665 - val_loss: 0.8295 - val_accuracy: 0.5016
Epoch 2/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.6252 - accuracy: 0.6880 - val_loss: 0.7395 - val_accuracy: 0.5338
Epoch 3/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.5871 - accuracy: 0.7042 - val_loss: 0.7485 - val_accuracy: 0.5434
Epoch 4/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.5726 - accuracy: 0.7108 - val_loss: 0.7624 - val_accuracy: 0.5370
Epoch 5/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.5524 - accuracy: 0.7281 - val_loss: 0.8926 - val_accuracy: 0.5273
Epoch 6/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.5306 - accuracy: 0.7377 - val_loss: 0.9414 - val_accuracy: 0.5370
Epoch 7/20
1670/1670 [==============================] - 4s 3ms/step - loss: 0.5163 - accu

In [168]:
batch_size = 5
epochs = 20
mag_fact = '100'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1752 samples, validate on 323 samples
Epoch 1/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.6684 - accuracy: 0.6707 - val_loss: 0.7844 - val_accuracy: 0.5820
Epoch 2/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.6189 - accuracy: 0.6992 - val_loss: 0.7431 - val_accuracy: 0.5511
Epoch 3/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.6138 - accuracy: 0.6992 - val_loss: 0.7098 - val_accuracy: 0.5697
Epoch 4/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.5808 - accuracy: 0.7135 - val_loss: 0.8446 - val_accuracy: 0.5820
Epoch 5/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.5683 - accuracy: 0.7220 - val_loss: 0.7268 - val_accuracy: 0.5635
Epoch 6/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.5570 - accuracy: 0.7329 - val_loss: 0.7380 - val_accuracy: 0.5418
Epoch 7/20
1752/1752 [==============================] - 5s 3ms/step - loss: 0.5481 - accu

In [169]:
batch_size = 5
epochs = 50
mag_fact = '200'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1688 samples, validate on 315 samples
Epoch 1/50
1688/1688 [==============================] - 5s 3ms/step - loss: 0.6415 - accuracy: 0.6890 - val_loss: 0.9638 - val_accuracy: 0.5651
Epoch 2/50
1688/1688 [==============================] - 4s 3ms/step - loss: 0.6179 - accuracy: 0.6967 - val_loss: 0.8257 - val_accuracy: 0.5651
Epoch 3/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5915 - accuracy: 0.7085 - val_loss: 0.7841 - val_accuracy: 0.5429
Epoch 4/50
1688/1688 [==============================] - 4s 3ms/step - loss: 0.5850 - accuracy: 0.7038 - val_loss: 0.7858 - val_accuracy: 0.4444
Epoch 5/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5791 - accuracy: 0.7062 - val_loss: 0.7210 - val_accuracy: 0.5714
Epoch 6/50
1688/1688 [==============================] - 4s 3ms/step - loss: 0.5647 - accuracy: 0.7227 - val_loss: 0.7451 - val_accuracy: 0.5302
Epoch 7/50
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5435 - accu

In [171]:
batch_size = 5
epochs = 20
mag_fact = '400'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1502 samples, validate on 287 samples
Epoch 1/20
1502/1502 [==============================] - 4s 3ms/step - loss: 0.6816 - accuracy: 0.6571 - val_loss: 0.9149 - val_accuracy: 0.5122
Epoch 2/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.6656 - accuracy: 0.6571 - val_loss: 0.8789 - val_accuracy: 0.5087
Epoch 3/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.6267 - accuracy: 0.6824 - val_loss: 0.7433 - val_accuracy: 0.5192
Epoch 4/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.6304 - accuracy: 0.6858 - val_loss: 0.7362 - val_accuracy: 0.5157
Epoch 5/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.6034 - accuracy: 0.6877 - val_loss: 0.8922 - val_accuracy: 0.5157
Epoch 6/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.5516 - accuracy: 0.7237 - val_loss: 0.7420 - val_accuracy: 0.5122
Epoch 7/20
1502/1502 [==============================] - 4s 2ms/step - loss: 0.5360 - accu

In [172]:
batch_size = 10
epochs = 50
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1670 samples, validate on 311 samples
Epoch 1/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.6400 - accuracy: 0.6808 - val_loss: 0.7461 - val_accuracy: 0.5563
Epoch 2/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.6182 - accuracy: 0.6928 - val_loss: 0.7690 - val_accuracy: 0.5498
Epoch 3/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.6056 - accuracy: 0.6982 - val_loss: 0.7274 - val_accuracy: 0.5498
Epoch 4/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.5786 - accuracy: 0.7144 - val_loss: 0.7818 - val_accuracy: 0.5531
Epoch 5/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.5680 - accuracy: 0.7126 - val_loss: 0.7491 - val_accuracy: 0.5498
Epoch 6/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.5530 - accuracy: 0.7275 - val_loss: 0.8442 - val_accuracy: 0.5531
Epoch 7/50
1670/1670 [==============================] - 3s 2ms/step - loss: 0.5504 - accu

In [175]:
def merge_model(mag):
    ############################################################################################
    # Input 1 (Nuclei)
    ############################################################################################
    model_1 = Sequential()
    model_1.add(Flatten(input_shape= (256, 256, 1)))
    model_1.add(Dropout(0.5))
    model_1.add(Dense(256, activation='sigmoid'))
    model_1.add(Dropout(0.5))
    model_1.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 2 (PCA)
    ############################################################################################
    PCA_input = Input(shape=(12,))
    model_2 = Sequential()
    model_2.add(Dense(512, input_dim=12))
    model_2.add(Activation('relu'))
    model_2.add(Dropout(0.5))
    model_2.add(Dense(256))
    model_2.add(Activation('sigmoid'))
    model_2.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
    ############################################################################################
    # Input 3 (VGG)
    ############################################################################################
    model_3 = Sequential()
    model_3.add(Flatten(input_shape= (7, 7, 512)))
    model_3.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

    merge_layer = concatenate([model_1.output, model_2.output, model_3.output])
    dnn_output = Dense(1, activation="sigmoid")(merge_layer)
    model_concat = Model(inputs=[model_1.input, model_2.input, model_3.input], outputs=dnn_output)
    return model_concat

In [176]:
batch_size = 10
epochs = 50
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = train_merge_model(mag_fact)

Train on 1670 samples, validate on 311 samples
Epoch 1/50
1670/1670 [==============================] - 11s 7ms/step - loss: 0.6453 - accuracy: 0.6802 - val_loss: 0.7186 - val_accuracy: 0.5563
Epoch 2/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.6208 - accuracy: 0.6880 - val_loss: 0.7259 - val_accuracy: 0.5531
Epoch 3/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.5948 - accuracy: 0.6994 - val_loss: 0.7899 - val_accuracy: 0.5659
Epoch 4/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.5790 - accuracy: 0.7102 - val_loss: 0.8043 - val_accuracy: 0.5466
Epoch 5/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.5783 - accuracy: 0.6994 - val_loss: 0.8172 - val_accuracy: 0.5531
Epoch 6/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.5585 - accuracy: 0.7198 - val_loss: 0.7426 - val_accuracy: 0.5531
Epoch 7/50
1670/1670 [==============================] - 11s 6ms/step - loss: 0.5489

In [177]:
def retrain(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    Nuc_train, Nuc_val, Nuc_test = Nuclei_feat(mag)
    PCA_train, PCA_val, PCA_test = PCA_feat(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)

    
    model_name = 'Final_concat_Model_' + mag
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")
    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(x = [Nuc_train, PCA_train, VGG_train], y = train_labels,
             validation_data = ([Nuc_val, PCA_val, VGG_val], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)
    
    test_score = model.evaluate(x = [Nuc_test, PCA_test, VGG_test], y = test_labels)
    print('\n', 'BreakHis Test accuracy:', test_score[1])

    model_name = 'Final_concat_Model_' + mag
    model_json = model.to_json()
    with open(model_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_name + '.h5')
    return model

In [178]:
batch_size = 10
epochs = 100
mag_fact = '40'
model_name = 'model_' + mag_fact
model_name = retrain(mag_fact)

Loaded model from disk
Train on 1670 samples, validate on 311 samples
Epoch 1/100
1670/1670 [==============================] - 11s 6ms/step - loss: 1.2251 - accuracy: 0.6533 - val_loss: 1.4969 - val_accuracy: 0.4759
Epoch 2/100
1670/1670 [==============================] - 10s 6ms/step - loss: 1.3494 - accuracy: 0.6431 - val_loss: 1.7575 - val_accuracy: 0.5434
Epoch 3/100
1670/1670 [==============================] - 10s 6ms/step - loss: 1.1625 - accuracy: 0.6707 - val_loss: 2.0592 - val_accuracy: 0.5531
Epoch 4/100
1670/1670 [==============================] - 10s 6ms/step - loss: 0.8943 - accuracy: 0.7168 - val_loss: 2.0697 - val_accuracy: 0.5563
Epoch 5/100
1670/1670 [==============================] - 11s 6ms/step - loss: 0.8619 - accuracy: 0.7246 - val_loss: 4.7503 - val_accuracy: 0.5595
Epoch 6/100
1670/1670 [==============================] - 10s 6ms/step - loss: 0.8509 - accuracy: 0.7311 - val_loss: 1.3510 - val_accuracy: 0.5434
Epoch 7/100
1670/1670 [==============================]

In [179]:
def merging_retrain(mag):
    train_dir, val_dir, test_dir, nb_train, nb_val, nb_test, train_labels, val_labels, test_labels = mag_conn(mag)
    Nuc_train, Nuc_val, Nuc_test = Nuclei_feat(mag)
    PCA_train, PCA_val, PCA_test = PCA_feat(mag)
    VGG_train, VGG_val, VGG_test = VGG_feat(mag)

    
    model_name = 'Merge_concat_Model'
    # load json and create model
    json_file = open(model_name + '.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(model_name + '.h5')
    print("Loaded model from disk")
    # evaluate loaded model on test data
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(x = [Nuc_train, PCA_train, VGG_train], y = train_labels,
             validation_data = ([Nuc_val, PCA_val, VGG_val], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)
    
    test_score = model.evaluate(x = [Nuc_test, PCA_test, VGG_test], y = test_labels)
    print('\n', 'BreakHis Test accuracy:', test_score[1])

    model_name = 'Merge_concat_Model' + mag
    model_json = model.to_json()
    with open(model_name + '.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(model_name + '.h5')
    return model

In [180]:
batch_size = 10
epochs = 100
mag_fact = '400'
remodel = merging_retrain(mag_fact)

batch_size = 10
epochs = 100
mag_fact = '200'
remodel = merging_retrain(mag_fact)

batch_size = 10
epochs = 100
mag_fact = '100'
remodel = merging_retrain(mag_fact)


Loaded model from disk
Train on 1502 samples, validate on 287 samples
Epoch 1/100
1502/1502 [==============================] - 10s 7ms/step - loss: 4.3673 - accuracy: 0.5706 - val_loss: 8.7797 - val_accuracy: 0.4878
Epoch 2/100
1502/1502 [==============================] - 10s 6ms/step - loss: 4.3858 - accuracy: 0.5832 - val_loss: 5.2159 - val_accuracy: 0.5226
Epoch 3/100
1502/1502 [==============================] - 10s 6ms/step - loss: 4.0838 - accuracy: 0.5892 - val_loss: 12.3782 - val_accuracy: 0.5122
Epoch 4/100
1502/1502 [==============================] - 10s 6ms/step - loss: 3.4446 - accuracy: 0.6165 - val_loss: 13.8957 - val_accuracy: 0.5122
Epoch 5/100
1502/1502 [==============================] - 10s 6ms/step - loss: 3.9393 - accuracy: 0.6079 - val_loss: 9.7356 - val_accuracy: 0.4878
Epoch 6/100
1502/1502 [==============================] - 10s 6ms/step - loss: 3.6195 - accuracy: 0.6152 - val_loss: 13.8772 - val_accuracy: 0.4843
Epoch 7/100
1502/1502 [============================